In [1]:
"""Set up connection to Google Drive."""

# import Drive helper and mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Suppress output of this cell
%%capture

"""Install and import dependencies."""

# Install and import spotipy to access Spotify API
!pip install spotipy
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Import configparser for loading configuration file to call Spotify API
import configparser

# Import pandas for dataframes
import pandas as pd 

# Import matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline 

# Import numpy to help with plots
import numpy as np

# Import scikit-learn for scaling and k-means clustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [0]:
"""Define path variables."""

# Main path in Google Drive
home_path = '/content/drive/My Drive/Colab Notebooks'

# Path to the Spotify configuration file
config_file_path = f'{home_path}/bin/spotify_client_config.txt'

# Path to save data
data_path = f'{home_path}/Data/Spotify audio features'

In [4]:
def init_spotify_client(config_file_path: str):
  """
  Initialise Spotify client to use Spotify API.
  
  Args:
      config_file_path: The path to the Spotify config file.
      
  Returns: 
      A Spotify client instance.
  """
  # Load the Spotify client configuration file
  config = configparser.ConfigParser()
  config.read(config_file_path)
  
  print(f'Loaded config file with settings for: {config.sections()}')

  # Use configuration to set up Spotify client authorisation
  client_credentials_manager = \
  SpotifyClientCredentials(client_id=config['developer.spotify.com']['cid'], \
                           client_secret=config['developer.spotify.com']['secret'])

  # Initialise the Spotify client
  sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
  
  print('Authorised Spotify client has been initialised')
  
  return sp


# Initialise the Spotify client
sp = init_spotify_client(config_file_path)

Loaded config file with settings for: ['developer.spotify.com']
Authorised Spotify client has been initialised


In [31]:
def get_album_info(sp, artist_url: str, album_type: str, request_limit: int):
  """
  Get album information for an artist on Spotify, and return them in a list.
  
  Args:
      sp: The Spotify client instance.
      artist_url: The Spotify URL of an artist.
      album_type: The album type to request.
      request_limit: The max number of records to return from the API request.
      
  Returns: 
      A list containing the names and Spotify URLs of albums for the artist
  """
  
  # Get the artist's album information from Spotify
  artist_album_info = sp.artist_albums(artist_url, album_type=album_type, limit=request_limit)
  
  # Get the list of albums
  album_list = artist_album_info['items']
  
  # Count the number of albums 
  num_albums = len(album_list)
  
  # Initialise empty list
  output_list = []
  
  # Check that there is at least 1 album
  if num_albums > 0:
    # Iterate over all albums
    for album_idx in range(0, num_albums):
      # Check that the album is the global version
      if album_list[album_idx]['available_markets'][0] != 'JP':
        # Record the album name and Spotify URL
        output_list.append([
            album_list[album_idx]['name'],
            album_list[album_idx]['external_urls']['spotify']
        ])
  
  print(f'Got {len(output_list)} album(s) of type "{album_type}"')
  
  return output_list


def get_artist_albums(sp, artist_url: str):
  """
  Get the names and Spotify URLs of all albums for an artist on Spotify, and return them in a list.
  
  Args:
      sp: The Spotify client.
      artist_url: The Spotify URL of an artist.
      
  Returns: 
      A list containing the album names and Spotify URLs for the artist
  """
  
  # Set limit to 50, which is the maximum
  request_limit = 50
  
  # Get all album information from Spotify
  artist_album_info = get_album_info(sp, artist_url, 'album', request_limit)
  artist_single_info = get_album_info(sp, artist_url, 'single', request_limit)
  artist_compilation_info = get_album_info(sp, artist_url, 'compilation', request_limit)
  
  all_album_info = artist_album_info + artist_single_info + artist_compilation_info
  
  print(f'Total number of albums: {len(all_album_info)}')
  
  return all_album_info
  
  
# Oku Hanako's Spotify URL
oku_hanako_url = 'https://open.spotify.com/artist/2tOwqfTtAMswbLySSaTRYR'

# Get album names and URLs
all_album_info = get_artist_albums(sp, oku_hanako_url)

Got 14 album(s) of type "album"
Got 22 album(s) of type "single"
Got 0 album(s) of type "compilation"
Total number of albums: 36


In [0]:
def get_album_tracks(sp, album_url: str):
  """
  Get IDs and names of all tracks in the album, and return them in a list.
  
  :param sp: Spotify client
  :param str album_url: URL of album on Spotify
  :returns: List containing IDs and names of tracks in the album
  """
  
  # Get album information from Spotify
  album = sp.album(album_url)
  num_tracks = album['total_tracks']
  album_name = album['name']
  
  print(f'Found {num_tracks} in {album_name}')
  
  # Initialise empty list
  track_list = []
  
  print(f'Start extracting track IDs and names from {album_name}')

  # Iterate over all tracks
  for track_idx in range(0, num_tracks):
    # Record the track ID and name
    track_list.append([
        album['tracks']['items'][track_idx]['id'],
        album['tracks']['items'][track_idx]['name']
    ])

  print(f'Finished extracting track IDs and names from {album_name}')
    
  return track_list


